In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.cif import CifData
from aiida.orm.calculation import Calculation

import ase.io
from ase.build import bulk

from apps.lsmo.structure_browser import StructureBrowser

import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

%matplotlib notebook


In [ ]:
import collections

def dict_merge(dct, merge_dct):
    """ Recursive dict merge. Inspired by :meth:``dict.update()``, instead of
    updating only top-level keys, dict_merge recurses down into dicts nested
    to an arbitrary depth, updating keys. The ``merge_dct`` is merged into
    ``dct``.
    :param dct: dict onto which the merge is executed
    :param merge_dct: dct merged into dct
    :return: None
    """
    for k, v in merge_dct.iteritems():
        if (k in dct and isinstance(dct[k], dict)
                and isinstance(merge_dct[k], collections.Mapping)):
            dict_merge(dct[k], merge_dct[k])
        else:
            dct[k] = merge_dct[k]

In [ ]:
atoms = None
node = False
structures = [("select structure",{"status":False})]

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, atoms, node
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    if node is False:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

## Step 1: Select Structure

In [ ]:
def on_struct_change(c):
    global atoms, node
    node = struct_browser.results.value
    refresh_structure_view()
    if not node:
        return
    cell_params.value = "Unit cell <br />a = {:.2f} {:.2f} {:.2f} <br />b = {:.2f} {:.2f} {:.2f} <br />c = {:.2f} {:.2f} {:.2f}".format(
        atoms.cell[0][0], atoms.cell[0][1], atoms.cell[0][2],
        atoms.cell[1][0], atoms.cell[1][1], atoms.cell[1][2],
        atoms.cell[2][0], atoms.cell[2][1], atoms.cell[2][2])

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()
cell_params = ipw.HTML("Cell: ")
clear_output()
display(ipw.VBox([struct_browser, viewer, cell_params]))

## Step 2: Specify the parameters and run Isotherm calculation with RASPA

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm import CalculationFactory, DataFactory
from aiida.orm.utils import load_node
from aiida.orm.code import Code
from aiida.orm.data.base import Str
from aiida.orm.data.array import ArrayData
from aiida.orm.data.upf import UpfData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData

from aiida.common.exceptions import NotExistent
from aiida.work.run import run, submit

from apps.lsmo.workflows.isotherm import Isotherm, IsothermSettings


In [ ]:
def get_code_options(plugin_classes):
    """
    Return AiiDA codes using a specific set of plugins
    
    :param plugin_classes: a dictionary of the type
      {'pw': 'quantumespresso.pw', 'ph': 'quantumespresso.ph'}
      where the key is a label and the value is the plugin to check for.
      It will return the set of codes that exist on the same machine.
    """
    from aiida.orm.querybuilder import QueryBuilder
    from aiida.orm import Code, Computer
    from aiida.backends.utils import get_automatic_user
    
    current_user = get_automatic_user()
    
    qb = QueryBuilder()
    qb.append(Computer,
          filters={'enabled': True},
          project=['*'], tag='computer')
    ordered_tags = []
    for tag, plugin_class in plugin_classes.iteritems():
        ordered_tags.append(tag)
        qb.append(Code,
          filters={'attributes.input_plugin': {'==': plugin_class},
                   'extras.hidden': {"~==": True}
            },
            project='label', tag='{}code'.format(tag), has_computer='computer')
    all_results = qb.all()
    # Filter in python only the ones that are actually user_configured
    # codeset[0] is the computer
    # codeset[1:] are the various code names, depending on the ones asked in input
    return [{tag: "{}@{}".format(codename, codeset[0].name) for codename, tag in zip(codeset[1:], ordered_tags)} 
            for codeset in all_results 
            if codeset[0].is_user_configured(current_user) and codeset[0].is_user_enabled(current_user)]

def get_code_dropdown(classes):
    """
    This function returns a group containing a dropdown list to select a
    valid available Quantum ESPRESSO pw.x code.

    To use it::

      code_group = get_code_pwonly_dropdown()


    You can later retrieve the value as follows::
   
      from IPython.display import display
      code_group = get_code_pwonly_dropdown()
      display(code_group)

    If this is None, then no code was found.
    Otherwise it will be a dictionary, where the only available key
    is 'pw' and the value is the code name, so you can get the code as::

       code_name = code_names['pw']
       code = Code.get_from_string(code_name)
    """
    import ipywidgets as ipw

    code_options_full = None
    in_codename = ipw.Dropdown(options=[], disabled=True)

    code_options_full = get_code_options(plugin_classes=classes)
    code_strings = ["{}".format(code_option[classes.keys()[0]]) 
        for code_option in code_options_full]  
        
    if code_options_full is None:
        in_codename.options=[["Error while retrieving the list of codes", None]]
        in_codename.disabled=True
        in_codename.value = None
    elif not code_options_full:
        in_codename.options = [["No AiiDA codes configured yet", None]]
        in_codename.disabled = True
        in_codename.value = None
    else:
        code_options = zip(code_strings, code_options_full)
        in_codename.options=code_options
        in_codename.disabled = False
        # Set default value (first entry)
        in_codename.value = code_options[0][1]    
                
    code_group = ipw.HBox(
        [
            ipw.Label(value="Select a code:"), 
            in_codename,
        ])

    return code_group

code_group_cp2k = get_code_dropdown(classes={'cp2k': 'cp2k'})
code_group_ddec = get_code_dropdown(classes={'ddec': 'ddec'})
code_group_cp2k.children[1].disabled = True
code_group_ddec.children[1].disabled = True

code_group_raspa = get_code_dropdown(classes={'raspa': 'raspa'})
code_group_zeopp = get_code_dropdown(classes={'zeopp.network': 'zeopp.network'})

In [ ]:
def on_charge_checkbox_change(c):
    if c.get('owner').value:
        code_group_cp2k.children[1].disabled = False
        code_group_ddec.children[1].disabled = False
        use_smearing.disabled = False
    else:
        code_group_cp2k.children[1].disabled = True
        code_group_ddec.children[1].disabled = True
        use_smearing.disabled = True


def on_click_submit(b):
    btn_submit.disabled = True
    if node is False:
        message.value = "Please select a structure"
        btn_submit.disabled = False
        return None
    message.value = 'Please wait, the isotherm is being computed.'

    options = {
        "resources": {
            "num_machines": 1,
            "tot_num_mpiprocs": 1,
            "num_mpiprocs_per_machine": 1,
        },
        "max_wallclock_seconds": 5 * 60 * 60,
        "withmpi": False,
    }
    pressures = ArrayData()
    pressures.set_array("pressures", np.array([0.001e5,
                                               0.01e5,
                                               0.1e5,
                                               0.2e5,
                                               0.4e5,
                                               0.6e5,
                                               0.8e5,
                                               1e5]))

    with submit_out:
        outputs = run(
            Isotherm,
            probe_molecule=ParameterData(dict=settings.probe_molecule.value),
            cp2k_parameters=ParameterData(dict=settings.return_cp2k_parameters(node, smearing=use_smearing.value)),
            ddec_parameters=ParameterData(dict=settings.return_ddec_parameters()),
            raspa_parameters=ParameterData(dict=settings.return_raspa_parameters(node)),
            pressures=pressures,
            structure=node,
            cp2k_codename=Str(code_group_cp2k.children[1].value['cp2k']),
            ddec_codename=Str(code_group_ddec.children[1].value['ddec']),
            zeopp_codename=Str(code_group_zeopp.children[1].value['zeopp.network']),
            raspa_codename=Str(code_group_raspa.children[1].value['raspa']),
            _options=options,
            _interactive=True,
            _usecharges = use_charges.value,
        )

        message.value = "Final results of the \"isotherm\" workchain:"
        print ("{pressure:12}  {loading_average:12}".format(pressure="pressure [bar]",
                                                            loading_average="Loading average [molecules/unit cell]"))
        print ("{}".format("-"*26))
        for p in outputs["result"].get_dict()['isotherm']:
            print ("{volume:>12.5f}  {energy:>12.5f}".format(volume=p[0], energy=p[1]))
            btn_submit.disabled = False


btn_submit = ipw.Button(description='Compute Isotherm')
btn_submit.on_click(on_click_submit)
message = ipw.HTML('')
submit_out = ipw.Output()

use_charges = ipw.Checkbox(
    value=False,
    description='Use charges',
    disabled=False
)

use_smearing = ipw.Checkbox(
    value=False,
    description='Use smearing',
    disabled=True
)
use_charges.observe(on_charge_checkbox_change, names='value')

code_group_cp2k

settings = IsothermSettings()

display(settings.settings_panel(),
        ipw.HBox([code_group_zeopp,
                  code_group_raspa,
                  btn_submit]),
        ipw.HBox([use_charges,
                  use_smearing]),
        ipw.HBox([code_group_cp2k,
                  code_group_ddec]),
        submit_out,
        message
       )